## Librerias

In [9]:
import pyodbc
import pandas as pd
import numpy as np
import os

## Pruebas de funcionamiento

**Funcinamiento Driver**

In [2]:
try:
    conn_str = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=master;Trusted_Connection=yes"
    conn = pyodbc.connect(conn_str)
    print("✅ Conexión exitosa a SQL Server")
    conn.close()
except Exception as e:
    print(f" Error de conexión: {e}")


✅ Conexión exitosa a SQL Server


**Importacion de las Consultas SQL**

In [3]:
BASE_DIR = os.getcwd()  # Devuelve el directorio actual donde se ejecuta el script
print(f" Directorio actual: {BASE_DIR}")

#Construir la ruta de la carpeta `queries`
SQL_QUERIES_PATH = os.path.join(BASE_DIR, "queries")
print(f" Ruta esperada para queries: {SQL_QUERIES_PATH}")

#  Verificar si encuentra la cerpta de las queries
if os.path.exists(SQL_QUERIES_PATH):
    print(" Carpeta encontrada correctamente.")
else:
    print(" ERROR: La carpeta no existe")


 Directorio actual: c:\Users\marti\gd_clv-6\data_migration
 Ruta esperada para queries: c:\Users\marti\gd_clv-6\data_migration\queries
 Carpeta encontrada correctamente.


In [5]:
if os.path.exists(SQL_QUERIES_PATH):
    sql_files = [f for f in os.listdir(SQL_QUERIES_PATH) if f.endswith(".sql")]
    print(f" Archivos SQL encontrados: {sql_files}")
else:
    print(" ERROR: No se encontró la carpeta `queries/`.")


 Archivos SQL encontrados: ['cliente.sql', 'fact_table.sql', 'geog.sql', 'prod.sql', 'tiempo.sql']


## Exportación Tablas Azure->Local

In [10]:

# Rutas de conexión Azure
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

#Conexión Azure
azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# Rutas SSMS
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
# Conexión a SQL Server Local
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

#  Ruta de la carpeta donde están las consutlas para modelo dimensional 
SQL_QUERIES_PATH = os.path.join(os.getcwd(), "queries")

#  Obtener lista de archivos SQL
sql_files = [f for f in os.listdir(SQL_QUERIES_PATH) if f.endswith(".sql")]

# Verificar si hay consultas en la carpeta
if not sql_files:
    print(" ERROR: No se encontraron archivos .sql en la carpeta.")
else:
    print(f" Encontrados {len(sql_files)} archivos SQL.")

#  Conectar a Azure SQL
try:
    print("1.Conectando a Azure SQL")
    conn_azure = pyodbc.connect(azure_conn_str)
    cursor_azure = conn_azure.cursor()

    print("2.Conectando a SQL Server Local")
    conn_local = pyodbc.connect(local_conn_str)
    cursor_local = conn_local.cursor()

    for sql_file in sql_files:
        file_path = os.path.join(SQL_QUERIES_PATH, sql_file)

        print(f" Procesando archivo: {sql_file}")

        # Leer el contenido del archivo SQL
        with open(file_path, 'r', encoding='utf-8') as file:
            sql_query = file.read()

        # Ejecutar consulta en Azure SQL
        print(f" Ejecutando consulta en Azure SQL para {sql_file}")
        df = pd.read_sql(sql_query, conn_azure)

        if df.empty:
            print(f" No hay datos para {sql_file}.No se creará la tabla en SQL Server Local.")
            continue

        print(f" Datos extraídos: {df.shape[0]} filas.")

        # Convertir NaN a 0
        df = df.fillna(0)

        # Ajustar tipos de datos
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)

        # Nombre de la tabla local (sin extensión)
        table_name = os.path.splitext(sql_file)[0]

        # Eliminar tabla si ya existe
        drop_table_sql = f"DROP TABLE IF EXISTS {table_name}"
        cursor_local.execute(drop_table_sql)
        conn_local.commit()

        print(f" Tabla {table_name} eliminada si existía.")

        # Crear la tabla con los tipos de datos ajustados
        create_table_sql = f"""
        CREATE TABLE {table_name} (
            {', '.join([
                f'[{col}] FLOAT' if df[col].dtype == np.float32 
                else f'[{col}] INT' if df[col].dtype == np.int32 
                else f'[{col}] NVARCHAR(255)' for col in df.columns
            ])}
        );
        """
        cursor_local.execute(create_table_sql)
        conn_local.commit()

        print(f"✅ Tabla {table_name} creada en SQL Server Local.")

        # Insertar datos en la tabla
        placeholders = ', '.join(['?' for _ in df.columns])
        insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"

        cursor_local.fast_executemany = True
        cursor_local.executemany(insert_sql, df.values.tolist())
        conn_local.commit()

        print(f" {df.shape[0]} filas insertadas en {table_name}.")

except Exception as e:
    print(f"ERROR: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n ¡Proceso completado!")


 Encontrados 5 archivos SQL.
1.Conectando a Azure SQL
2.Conectando a SQL Server Local
 Procesando archivo: cliente.sql
 Ejecutando consulta en Azure SQL para cliente.sql


C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1760372507.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


 Datos extraídos: 44053 filas.
 Tabla cliente eliminada si existía.
✅ Tabla cliente creada en SQL Server Local.
 44053 filas insertadas en cliente.
 Procesando archivo: fact_table.sql
 Ejecutando consulta en Azure SQL para fact_table.sql


C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1760372507.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


 Datos extraídos: 58049 filas.
 Tabla fact_table eliminada si existía.
✅ Tabla fact_table creada en SQL Server Local.
 58049 filas insertadas en fact_table.
 Procesando archivo: geog.sql
 Ejecutando consulta en Azure SQL para geog.sql
 Datos extraídos: 12 filas.
 Tabla geog eliminada si existía.
✅ Tabla geog creada en SQL Server Local.
 12 filas insertadas en geog.
 Procesando archivo: prod.sql
 Ejecutando consulta en Azure SQL para prod.sql


C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1760372507.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)
C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1760372507.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


 Datos extraídos: 404 filas.
 Tabla prod eliminada si existía.
✅ Tabla prod creada en SQL Server Local.
 404 filas insertadas en prod.
 Procesando archivo: tiempo.sql
 Ejecutando consulta en Azure SQL para tiempo.sql
 Datos extraídos: 3652 filas.


C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1760372507.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


 Tabla tiempo eliminada si existía.
✅ Tabla tiempo creada en SQL Server Local.
 3652 filas insertadas en tiempo.

 ¡Proceso completado!


## Comprobaciones Finales

En el siguiente código vamos a hacer una **comparación del número de registros** en las tablas de Azure con las tablas en local.Comparamos de esta forma:
| **Local Table (SSMS)** | **Azure Table**             |
|------------------------|-----------------------------|
| [dbo].[fact_table]   | [DATAEX].[001_sales]      |
| [dbo].[cliente]      | [DATAEX].[003_clientes]   |
| [dbo].[prod]         | [DATAEX].[006_producto]   |
| [dbo].[geog]        | [DATAEX].[011_tienda]     |




In [11]:
# Clave: nombre de la tabla en ssms
# Valor: nombre completo de la tabla en Azure ( por ejemplo, "[DATAEX].[001_sales]")
tables = {
    "[dbo].[fact_table]": "[DATAEX].[001_sales]",
    "[dbo].[cliente]":    "[DATAEX].[003_clientes]",
    "[dbo].[prod]":       "[DATAEX].[006_producto]",
    "[dbo].[geog]":       "[DATAEX].[011_tienda]"
}

try:
    print("Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    print("Conectando a SQL Server Local...")
    conn_local = pyodbc.connect(local_conn_str)
    
    for local_table, azure_table in tables.items():
        # Consulta para contar filas en Azure
        azure_query = f"SELECT COUNT(*) AS Total_Filas FROM {azure_table}"
        df_azure = pd.read_sql(azure_query, conn_azure)
        azure_count = df_azure['Total_Filas'].iloc[0]
        
        # Consulta para contar filas en la tabla local
        local_query = f"SELECT COUNT(*) AS Total_Filas FROM {local_table}"
        df_local = pd.read_sql(local_query, conn_local)
        local_count = df_local['Total_Filas'].iloc[0]
        
        print(f"Tabla local '{local_table}':")
        print(f"    Azure SQL ({azure_table}): {azure_count} filas")
        print(f"    SQL Local: {local_count} filas")
        
        if azure_count == local_count:
            print("✅ Los recuentos coinciden.\n")
        else:
            print(" Los recuentos NO coinciden.\n")
            
except Exception as e:
    print("ERROR:", e)
    
finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("¡No se ha perdido ningún dato por el camino :)!")

Conectando a Azure SQL...
Conectando a SQL Server Local...


C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1355445676.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_azure = pd.read_sql(azure_query, conn_azure)
C:\Users\marti\AppData\Local\Temp\ipykernel_25988\1355445676.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_local = pd.read_sql(local_query, conn_local)


Tabla local '[dbo].[fact_table]':
    Azure SQL ([DATAEX].[001_sales]): 58049 filas
    SQL Local: 58049 filas
✅ Los recuentos coinciden.

Tabla local '[dbo].[cliente]':
    Azure SQL ([DATAEX].[003_clientes]): 44053 filas
    SQL Local: 44053 filas
✅ Los recuentos coinciden.

Tabla local '[dbo].[prod]':
    Azure SQL ([DATAEX].[006_producto]): 404 filas
    SQL Local: 404 filas
✅ Los recuentos coinciden.

Tabla local '[dbo].[geog]':
    Azure SQL ([DATAEX].[011_tienda]): 12 filas
    SQL Local: 12 filas
✅ Los recuentos coinciden.

¡No se ha perdido ningún dato por el camino :)!
